In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print((torch.cuda._get_nvml_device_index(0)))

True
NVIDIA A100-SXM4-80GB
3


In [3]:
from langchain.callbacks.manager import CallbackManager
from langchain_community.vectorstores import ElasticsearchStore
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import multiprocessing
import os

In [4]:
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, ".."))
MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "hyperclova", "hyperclova-seed-text-1.5b-q4-k-m.gguf")

In [5]:
# base_system_prompt.txt 로드
with open("./prompts/system/base_system_prompt_v0.2.txt", "r", encoding="utf-8") as f:
    base_system_prompt = f.read()

# qa_prompt.txt 로드 
with open("./prompts/tasks/qa_prompt_v0.2.txt", "r", encoding="utf-8") as f:
    qa_prompt = f.read()

In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name="../ai_models/base_models/BGE-m3-ko",
    model_kwargs={'device': 'cuda:0'},
    encode_kwargs={'normalize_embeddings': True}
)

/tmp/ipykernel_1244096/1904420697.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [7]:
faiss_index_directory = "./faiss_index_directory"
vectorstore = FAISS.load_local(faiss_index_directory, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(    
    search_type="similarity_score_threshold", 
    search_kwargs={"score_threshold": 0.5, "k": 20}
)

In [8]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    model_path=MODEL_PATH,
    temperature=0.6,         # 약간 더 창의적인 응답 유도
    top_p=0.9,               # 너무 많은 토큰 허용하지 않도록 조정
    max_tokens=512,
    repeat_penalty=1.3,
    stop=["\n###","<<END>>"],
    presence_penalty=0.2,
    frequency_penalty=0.3, 
    top_k=45,
    callback_manager=callback_manager,
    verbose=True,
    n_ctx=2048,
    n_gpu_layers=-1,
    n_batch=512,
    device="cuda",
    f16_kv=True
)

# llm = LlamaCpp(
#     model_path=MODEL_PATH,
#     temperature=0.6,
#     max_tokens=512,
#     top_p=1,
#     callback_manager=callback_manager,
#     verbose=True,
#     n_ctx=2048,  # 컨텍스트 길이
#     n_gpu_layers=-1,  # 모든 레이어를 GPU에서 실행 (-1은 전체 레이어)
#     n_batch=512,  # GPU 배치 크기
#     # n_threads=multiprocessing.cpu_count() - 1,
#     device="cuda",  # GPU 사용 설정
#     f16_kv=True  # GPU 메모리 최적화를 위한 FP16 사용
# )

/AIHUB/PCNRND/home/chatbot/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3607: UserWarning: WARNING! presence_penalty is not default parameter.
                presence_penalty was transferred to model_kwargs.
                Please confirm that presence_penalty is what you intended.
  if await self.run_code(code, result, async_=asy):
/AIHUB/PCNRND/home/chatbot/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3607: UserWarning: WARNING! frequency_penalty is not default parameter.
                frequency_penalty was transferred to model_kwargs.
                Please confirm that frequency_penalty is what you intended.
  if await self.run_code(code, result, async_=asy):
/AIHUB/PCNRND/home/chatbot/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3607: UserWarning: WARNING! device is not default parameter.
                device was transferred to model_kwargs.
                Please confirm that device is what you intended.
  

In [9]:
# template=f"{base_system_prompt}\n{{context}}\n\n{qa_prompt}"
# prompt_template = PromptTemplate.from_template(template=template) # template_format="f-string"
template = f"""{base_system_prompt}

{{context}}

{qa_prompt}"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)


In [10]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,                     # llama-cpp나 OpenAI 등 langchain-compatible LLM
    retriever=retriever,         # langchain-compatible retriever
    chain_type="stuff",          # "stuff", "map_reduce", "refine" 중 선택
    chain_type_kwargs={"prompt": prompt_template}  # PromptTemplate을 전달
)


In [11]:
final_result = rag_chain.invoke("초거대 AI 모델이란?").get("result")

초거대 AI 모델이란, 대규모의 데이터를 처리하는데 사용되는 모델을 의미합니다. 초 거대서AI모델은  대량의 데이터가 있는 경우에만 사용 가능하며 사용을 위해서는 반드시 필요한 조건에 부합해야 사용할 수 있습니다.

- 규모를 기준으로 삼습니다.
- 사용자의 의도를 파악하고 이를 준수하여 일을 처리할 때 다음과 같은 사항을 명세하게 기술한 내용을 주된 사항만 특정하지 않고 상세히 설명하거나 지정합니다. 

1. **질의 의도**는 초거대 AI 모델은 다음을 수행하기 위한 지침에 따르세요.

2. 적어도 3.

llama_perf_context_print:        load time =   34405.78 ms
llama_perf_context_print: prompt eval time =   34405.36 ms /   545 tokens (   63.13 ms per token,    15.84 tokens per second)
llama_perf_context_print:        eval time =    3534.66 ms /   135 runs   (   26.18 ms per token,    38.19 tokens per second)
llama_perf_context_print:       total time =   38198.18 ms /   680 tokens


In [12]:
print(final_result)

초거대 AI 모델이란, 대규모의 데이터를 처리하는데 사용되는 모델을 의미합니다. 초 거대서AI모델은  대량의 데이터가 있는 경우에만 사용 가능하며 사용을 위해서는 반드시 필요한 조건에 부합해야 사용할 수 있습니다.

- 규모를 기준으로 삼습니다.
- 사용자의 의도를 파악하고 이를 준수하여 일을 처리할 때 다음과 같은 사항을 명세하게 기술한 내용을 주된 사항만 특정하지 않고 상세히 설명하거나 지정합니다. 

1. **질의 의도**는 초거대 AI 모델은 다음을 수행하기 위한 지침에 따르세요.

2. 적어도 3.


In [13]:
vectorstore.similarity_search("초거대 AI 모델이란?", k=20)

[Document(id='d5b0a41e-a911-414a-8612-0974f2d7b847', metadata={'producer': 'Hancom PDF 1.3.0.547', 'creator': 'Hwp 2024 13.0.0.711', 'creationdate': '2025-02-14T14:24:06+09:00', 'author': 'Kevin', 'moddate': '2025-02-14T14:24:06+09:00', 'pdfversion': '1.4', 'source': '../data/pdf/IS-198 글로벌 초거대 AI 모델 현황 분석(2024년 조사).pdf', 'total_pages': 21, 'page': 4, 'page_label': '5'}, page_content='.\x00연구\x00개요□초거대 AI 모델ㅇ초거대 AI 모델이란, 대규모의 컴퓨팅 인프라를 바탕으로 방대한 데이터를 학습하여 인간처럼 종합적인 인지·판단·추론이 가능해진 ‘큰 규모’의 AI 모델을 의미함(봉강호, 2024)1)\xad특정 목적에 따라 개별의 데이터를 수집·학습하여 만들어지는 기존의 일반 AI는 학습된 과업(task)에 한하여 수행이 가능한 반면, 초거대 AI는 더욱 복잡하고 광범위한 분야에서 과업을 수행할 수 있음ㅇAI 모델 성능이 ‘모델의 규모(model size)’, ‘학습 데이터(dataset)’ 및 ‘학습 연산(training compute)’의 양에 따라 향상되는 경향성'),
 Document(id='bfa3c453-5ecc-4106-bdd8-60a4ddf6f5f8', metadata={'producer': 'Hancom PDF 1.3.0.547', 'creator': 'Hwp 2024 13.0.0.711', 'creationdate': '2025-02-14T14:24:06+09:00', 'author': 'Kevin', 'moddate': '2025-02-14T14:24:06+09:00', 'pdfversion': '1.4', 'source': '../